In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['PATH_TO_ENV'] = "~/projects/chatsky-llm-autoconfig/.env"

In [3]:
from dialogue2graph.pipelines.core.dialogue import DialogueMessage, Dialogue
import pickle
import numpy as np

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [9]:
from tqdm import tqdm
from dialogue2graph.metrics.no_llm_metrics.metrics import match_roles, is_correct_length
from augmentation_utils import (
    check_no_duplicates_all_dialogues, check_no_duplicates_one_dialogue, check_no_duplicates_one_uttr_list
    )

In [97]:
with open("../data/gen_dataset_augment_uttr-vars_8", "rb") as fp:
    data = pickle.load(fp)
len(data)

1

### Checking matching roles and length correctness

In [108]:
# errors = []
# failure_examples = []
# new_data = []

# for i, example in enumerate(tqdm(data)):
#     dialogues = [dial['messages'] for dial in example['dialogues']]
#     augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
#     true_count = 0

#     try:
#         for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
#             orig_dia = Dialogue.from_list(orig_dia)
#             aug_dia = Dialogue.from_list(aug_dia)
#             if is_correct_length(orig_dia, aug_dia):
#                 true_count += 1
#             else:
#                 failure_examples.append((i, j))
#     except Exception as e:
#         errors.append((i, j, e))

#     if true_count == len(augmented_dialogues):
#         new_data.append(example)

# failure_examples, errors

In [107]:
# errors = []
# failure_examples = []
# new_data = []

# for i, example in enumerate(tqdm(data)):
#     dialogues = [dial['messages'] for dial in example['dialogues']]
#     augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
#     true_count = 0

#     try:
#         for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
#             orig_dia = Dialogue.from_list(orig_dia)
#             aug_dia = Dialogue.from_list(aug_dia)
#             if match_roles(orig_dia, aug_dia):
#                 true_count += 1
#             else:
#                 failure_examples.append((i, j))
#     except Exception as e:
#         errors.append((i, j, e))

#     if true_count == len(augmented_dialogues):
#         new_data.append(example)

# failure_examples, errors

### Checking duplicates

In [98]:
new_data = []
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    for j, aug_dia in enumerate(augmented_dialogues):
        try:
            if not check_no_duplicates_one_uttr_list(aug_dia):
                failure_examples.append((i, j))
            else:
                true_count += 1
        except Exception as e:
            errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data.append(example)

100%|██████████| 1/1 [00:00<00:00, 3953.16it/s]


In [99]:
errors, failure_examples

([], [])

In [101]:
new_data = []
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    print(i)
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
    true_count = 0

    for j, aug_dia in enumerate(augmented_dialogues):
        try:
            if not check_no_duplicates_one_dialogue(aug_dia):
                failure_examples.append((i, j))
            else:
                true_count += 1
        except Exception as e:
            errors.append((i, j, e))

    if true_count == len(augmented_dialogues):
        new_data.append(example)

100%|██████████| 1/1 [00:00<00:00, 3813.00it/s]

0
common_elements: ['Medium, please.', 'Red, please.', 'Sure, it’s 123 Main St.']
common_elements: ["Sure, it's 123 Main St."]


In [102]:
print(errors, failure_examples)

[] [(0, 0), (0, 1)]


In [109]:
new_data = []
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    print(f'example {i}')
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        if not check_no_duplicates_all_dialogues(augmented_dialogues):
            failure_examples.append(i)
        else:
            new_data.append(example)
    except Exception as e:
        errors.append((i, e))
print(errors, failure_examples)

100%|██████████| 1/1 [00:00<00:00, 3382.50it/s]

example 0
common_elements: {'Hi! What assistance do you need today?', 'Red, please.', 'I’d like to make an order.', 'Would you like that in red or blue?', 'Awesome! Which product are you interested in?', 'Red is my choice.', 'Sure, it’s 123 Main St.', 'Medium, please.', 'Medium would be great.', 'I’d like it in red.', 'Do you want it in red or blue?', 'Hello! How can I help you today?', 'Perfect! What’s your shipping address?'}
[] [0]


In [106]:
augmented_dialogues[1]

[{'participant': 'assistant',
  'text': ['Hello! How can I help you today?',
   'Hi! What can I do for you today?',
   'Hey there! How may I assist you?']},
 {'participant': 'user',
  'text': ["I'm looking to place an order.",
   'I would like to make an order.',
   'I want to order something.']},
 {'participant': 'assistant',
  'text': ['Awesome! Which product are you looking for?',
   'Great choice! What item are you interested in?',
   'Fantastic! What would you like to order?']},
 {'participant': 'user',
  'text': ["I'd like to get a t-shirt.",
   'I want to order a t-shirt.',
   "I'm interested in a t-shirt."]},
 {'participant': 'assistant',
  'text': ['What size do you need?',
   "Could you let me know what size you'd like?",
   'Which size are you looking for?']},
 {'participant': 'user',
  'text': ["I'd like a medium size, please.",
   'Medium would be great, thanks.',
   'Please make it a medium.']},
 {'participant': 'assistant',
  'text': ['Would you prefer that in red or blu